# Example: Software Development Kit for the National Weather Service API
In this example, students will build a Software Development Kit (SDK) for the [National Weather Service API](https://www.weather.gov/documentation/services-web-api). We'll pull down weather data for Ithaca, NY, using the `(latitude, longitude)` coordinates `42.443961` and `-76.501881`. We'll use the `points` endpoint to get the grid data for this location and use this data for additional calls. 


## Setup
This example requires several external libraries and a function to compute the outer product. Let's download and install these packages and call our `Include.jl` file.

In [9]:
include("Include.jl");

## Task 1: Build a GridPoint Endpoint Model, Call the Weather API
In this task, we'll build a model of the Endpoint call that we want to do and then call the Weather API to get raw data back for the weather in Ithaca, NY.

Build a [mutable struct model](https://docs.julialang.org/en/v1/manual/types/#Composite-Types) of the `points` endpoint we want to call. This `struct` should have a field for each piece of information that we need to pass to the `API` with the same names as the `endpoint` (convention, not required)

Complete the implementation of the `MyWeatherGridPointEndpointModel` struct in `src/Types.jl`. This should be a mutable struct with fields for `latitude,` and `longitude.` It should have a constructor that takes `latitude` and `longitude` as keyword arguments, so can build the `struct` with a call like:

In [15]:
model = MyWeatherGridPointEndpointModel(latitude = 42.443962, longitude = -76.501884);

Next, let's set up [the Uniform Resource Locator (URL) string](https://en.wikipedia.org/wiki/Query_string), which we will call. This string will encode the resource we want (endpoint) and how to access it (protocol). We save the weather service URL in the `points_url_string::String` variable:

In [24]:
points_url_string = build("https://api.weather.gov", model)

"https://api.weather.gov/points/42.443962,-76.501884"

Now, let's make our first call!

In [27]:
result_points = MyWeatherGridPointEndpointModel(points_url_string);

In [29]:
result_points

Dict{String, Any} with 5 entries:
  "@context"   => Any["https://geojson.org/geojson-ld/geojson-context.jsonld", …
  "geometry"   => Dict{String, Any}("coordinates"=>Any[-76.5019, 42.444], "type…
  "id"         => "https://api.weather.gov/points/42.444,-76.5019"
  "properties" => Dict{String, Any}("county"=>"https://api.weather.gov/zones/co…
  "type"       => "Feature"

In [37]:
forecast_url = result_points["properties"]["forecastHourly"]

"https://api.weather.gov/gridpoints/BGM/44,70/forecast/hourly"

Finally, we have [the URL](https://en.wikipedia.org/wiki/Query_string), allowing us to get the weather forecast for Ithaca, NY. Let's call the Weather Service API again with the `forecast_url::String` URL and look at what comes back from the API:

In [47]:
result_forecast = MyWeatherForecastEndpointModel(forecast_url)

Dict{String, Any} with 4 entries:
  "@context"   => Any["https://geojson.org/geojson-ld/geojson-context.jsonld", …
  "geometry"   => Dict{String, Any}("coordinates"=>Any[Any[Any[-76.5195, 42.457…
  "properties" => Dict{String, Any}("units"=>"us", "generatedAt"=>"2024-11-23T1…
  "type"       => "Feature"

## Task 2: Let's do a better job an handling the forecast data coming back
In this task, we'll demonstrate a handler pattern in which the raw data from the API is processed before being presented to the caller. 
* __Why would we do this?__ We use a handler pattern if we want the data from the API to be in a more convenient form, for example [as a DataFrame instance](https://dataframes.juliadata.org/stable/), or we want to manipulate or process the data is some way. Ultimately, we use this pattern to make our lives easier when we use the data coming back from the API call.

In [57]:
result_forecast = MyWeatherForecastEndpointModel(forecast_url, handler = process_forecast_response_dataframe);

In [59]:
result_forecast

Row,startTime,endTime,isDayTime,temperature,temperatureUnit,windSpeed,windDirection,shortForecast
,String,String,Bool,Int64,String,String,String,String
1,2024-11-23T09:00:00-05:00,2024-11-23T10:00:00-05:00,true,41,F,14 mph,NW,Slight Chance Rain Showers
2,2024-11-23T10:00:00-05:00,2024-11-23T11:00:00-05:00,true,42,F,16 mph,NW,Slight Chance Rain Showers
3,2024-11-23T11:00:00-05:00,2024-11-23T12:00:00-05:00,true,44,F,16 mph,NW,Slight Chance Rain Showers
4,2024-11-23T12:00:00-05:00,2024-11-23T13:00:00-05:00,true,45,F,16 mph,NW,Slight Chance Rain Showers
5,2024-11-23T13:00:00-05:00,2024-11-23T14:00:00-05:00,true,45,F,16 mph,NW,Chance Rain Showers
6,2024-11-23T14:00:00-05:00,2024-11-23T15:00:00-05:00,true,45,F,17 mph,NW,Chance Rain Showers
7,2024-11-23T15:00:00-05:00,2024-11-23T16:00:00-05:00,true,46,F,16 mph,NW,Chance Rain Showers
8,2024-11-23T16:00:00-05:00,2024-11-23T17:00:00-05:00,true,45,F,16 mph,W,Chance Rain Showers
9,2024-11-23T17:00:00-05:00,2024-11-23T18:00:00-05:00,true,45,F,15 mph,W,Chance Rain Showers
